In [1]:
import rosbag, os, matplotlib, pickle
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from qsrlib.qsrlib import QSRlib, QSRlib_Request_Message
from qsrlib_io.world_trace import Object_State, World_Trace
from qsrlib.qsrlib import QSR_QTC_BC_Simplified
import datetime as dt
import numpy as np
import pandas as pd

os.chdir("/home/loz/QTC_Trajectory_HMMs/from_bags/")

In [2]:
# In[3]:
os.listdir("sim_bags")

['S7-i36-e15_2020-01-14-11-59-06.bag',
 'S1-i25-e59_2020-01-14-12-17-09.bag',
 'S3-i36-e15_2020-01-14-12-39-44.bag',
 'S6-i36-e15_2020-01-14-14-23-48.bag',
 'S1-i25-e59_2020-01-14-13-33-53.bag',
 'S2-i25-e35_2020-01-14-12-26-11.bag',
 'S8-i25-e45_2020-01-14-14-37-20.bag',
 'S8-i25-e45_2020-01-14-13-20-17.bag',
 'S4-i36-e15_2020-01-14-12-48-47.bag',
 'S2-i25-e35_2020-01-14-13-42-56.bag',
 'S1-i25-e35_2020-01-14-13-29-22.bag',
 'S2-i25-e59_2020-01-14-12-30-42.bag',
 'S2-i25-e59_2020-01-14-11-13-34.bag',
 'S7-i33-e40_2020-01-14-11-54-34.bag',
 'S3-i33-e40_2020-01-14-13-51-59.bag',
 'S5-i25-e45_2020-01-14-11-41-02.bag',
 'S4-i33-e40_2020-01-14-12-44-15.bag',
 'S7-i33-e40_2020-01-14-14-28-18.bag',
 'S1-i25-e35_2020-01-14-12-12-38.bag',
 'S1-i25-e35_2020-01-14-10-54-56.bag',
 'S7-i33-e40_2020-01-14-13-11-18.bag',
 'S4-i36-e15_2020-01-14-14-05-43.bag',
 'S3-i36-e15_2020-01-14-11-22-53.bag',
 'S2-i25-e48_2020-01-14-13-38-25.bag',
 'S5-i25-e45_2020-01-14-14-14-43.bag',
 'S5-i25-e45_2020-01-14-1

In [3]:
# In[4]:


sim_bags = os.listdir("sim_bags")
sim_bags

no_collide_bag_codes = [
    "S1-i25-e35",
    "S1-i25-e59",
    "S2-i25-e35",
    "S2-i25-e59",
    "S3-i33-e40",
    "S3-i36-e15",
    "S4-i33-e40",
    "S4-i36-e15",
    "S6-i33-e40",
    "S6-i36-e15",
    "S7-i33-e40",
    "S7-i36-e15"
]

sim_bags = [bag for bag in sim_bags if bag[:10] in no_collide_bag_codes]
sim_bags

['S7-i36-e15_2020-01-14-11-59-06.bag',
 'S1-i25-e59_2020-01-14-12-17-09.bag',
 'S3-i36-e15_2020-01-14-12-39-44.bag',
 'S6-i36-e15_2020-01-14-14-23-48.bag',
 'S1-i25-e59_2020-01-14-13-33-53.bag',
 'S2-i25-e35_2020-01-14-12-26-11.bag',
 'S4-i36-e15_2020-01-14-12-48-47.bag',
 'S2-i25-e35_2020-01-14-13-42-56.bag',
 'S1-i25-e35_2020-01-14-13-29-22.bag',
 'S2-i25-e59_2020-01-14-12-30-42.bag',
 'S2-i25-e59_2020-01-14-11-13-34.bag',
 'S7-i33-e40_2020-01-14-11-54-34.bag',
 'S3-i33-e40_2020-01-14-13-51-59.bag',
 'S4-i33-e40_2020-01-14-12-44-15.bag',
 'S7-i33-e40_2020-01-14-14-28-18.bag',
 'S1-i25-e35_2020-01-14-12-12-38.bag',
 'S1-i25-e35_2020-01-14-10-54-56.bag',
 'S7-i33-e40_2020-01-14-13-11-18.bag',
 'S4-i36-e15_2020-01-14-14-05-43.bag',
 'S3-i36-e15_2020-01-14-11-22-53.bag',
 'S7-i36-e15_2020-01-14-14-32-49.bag',
 'S2-i25-e59_2020-01-14-13-47-27.bag',
 'S7-i36-e15_2020-01-14-13-15-48.bag',
 'S6-i36-e15_2020-01-14-11-50-02.bag',
 'S6-i33-e40_2020-01-14-11-45-31.bag',
 'S1-i25-e59_2020-01-14-1

In [4]:
# In[5]:


sim_r_positions = []
sim_h_positions = []

for bag_path in sim_bags:
    bag_path = "sim_bags/"+bag_path
    bag = rosbag.Bag(bag_path)
    
    r_xs = []
    r_ys = []
    for topic, msg, t in bag.read_messages(topics=['/robot4/robot_pose']):
        r_xs.append(msg.pose.position.x)
        r_ys.append(msg.pose.position.y)    
    
    h_xs = []
    h_ys = []
    for topic, msg, t in bag.read_messages(topics=['/robot4/human_perception/tracked_persons']):
        try:
            h_xs.append(msg.tracks[0].pose.pose.position.x)
            h_ys.append(msg.tracks[0].pose.pose.position.y)
        except:
            pass
    
    bag.close()
    sim_r_positions.append([r_xs, r_ys])
    sim_h_positions.append([h_xs, h_ys])

In [5]:
# In[6]:


for bag_no in range(len(sim_bags)):

    matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
    matplotlib.rcParams['font.size'] = 30

    r_x = sim_r_positions[bag_no][0]
    r_y = sim_r_positions[bag_no][1]

    h_x = sim_h_positions[bag_no][0]
    h_y = sim_h_positions[bag_no][1]

    r_colour = np.array([[shade, 0, 0] for shade in np.linspace(50, 255, len(r_x))]) / 255.0
    h_colour = np.array([[0, 0, shade] for shade in np.linspace(50, 255, len(h_x))]) / 255.0

    r_size = np.linspace(10, 100, len(r_x))
    h_size = np.linspace(10, 100, len(h_x))

    plt.scatter(r_x, r_y, c=r_colour, s=r_size)
    
    # Get QTC_C sequence
    bag_path = "sim_bags/"+sim_bags[bag_no]
    bag = rosbag.Bag(bag_path)
    qtc_c_seq = []
    for topic, msg, t in bag.read_messages(topics=['/robot4/qtc_state_topics']):
        qtc_c_seq.append(msg.data)
    
    qtc_c_seq = [qtc for qtc in qtc_c_seq if qtc != '0,0,0,0']
    
    
    plt.title(sim_bags[bag_no])

    plt.scatter(h_x, h_y, c=h_colour, s=h_size)
    
    plt.savefig("sim_bag_plots/"+sim_bags[bag_no][:-4]+".png")
    plt.close()
    
#     print(sim_bags[bag_no]+":\n"+str(qtc_c_seq)+"\n")

S7-i36-e15_2020-01-14-11-59-06.bag:
['-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,-,-,0', '-,-,-,0', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,0,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,0']

S1-i25-e59_2020-01-14-12-17-09.bag:
['-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,-,+,-', '-,-,+,-', '-,-,+,-', '-,-,+,-', '

S7-i33-e40_2020-01-14-11-54-34.bag:
['-,0,0,0', '-,0,0,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0']

S3-i33-e40_2020-01-14-13-51-59.bag:
['+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '+,0,+,0', '

S7-i36-e15_2020-01-14-13-15-48.bag:
['-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,0,-,0', '-,-,-,0', '-,-,-,0', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,-,-,-', '-,0,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '+,+,-,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,-', '0,+,0,0']

S6-i36-e15_2020-01-14-11-50-02.bag:
['-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '

S6-i33-e40_2020-01-14-14-19-16.bag:
['-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,0,+,0', '-,-,+,0', '-,-,+,0', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '-,-,+,+', '+,-,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,+,+', '+,+,0,0']

S4-i33-e40_2020-01-14-11-27-29.bag:
['0,0,-,0', '0,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,0,-,0', '+,-,-,+', '+,-,-,+', '

In [6]:
# # In[7]:


# failed_bags = []
# for bag_no in range(len(sim_bags)):

#     matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
#     matplotlib.rcParams['font.size'] = 30

#     r_x = sim_r_positions[bag_no][0]
#     r_y = sim_r_positions[bag_no][1]

#     h_x = sim_h_positions[bag_no][0]
#     h_y = sim_h_positions[bag_no][1]

#     r_colour = np.array([[shade, 0, 0] for shade in np.linspace(50, 255, len(r_x))]) / 255.0
#     h_colour = np.array([[0, 0, shade] for shade in np.linspace(50, 255, len(h_x))]) / 255.0

#     r_size = np.linspace(10, 100, len(r_x))
#     h_size = np.linspace(10, 100, len(h_x))

#     plt.scatter(r_x, r_y, c=r_colour, s=r_size)
    
    
#     plt.title(sim_bags[bag_no])

#     plt.scatter(h_x, h_y, c=h_colour, s=h_size)
    
#     plt.show()
    
#     bag_success = raw_input("Was the interaction simulated sucessfully? N for no:\n")
#     print("\n")
#     if bag_success.upper() == "N":
#         failed_bags.append(sim_bags[bag_no])
#         print("The above simulation failed.\n")
#     else:
#         print("The above simulation passed.\n")
    
#     plt.close()

In [7]:
# In[ ]:

failed_bags = ['S3-i36-e15_2020-01-14-12-39-44.bag',
 'S7-i33-e40_2020-01-14-11-54-34.bag',
 'S1-i25-e35_2020-01-14-12-12-38.bag',
 'S3-i36-e15_2020-01-14-11-22-53.bag',
 'S6-i36-e15_2020-01-14-11-50-02.bag',
 'S3-i36-e15_2020-01-14-13-56-39.bag']
failed_bags

['S3-i36-e15_2020-01-14-12-39-44.bag',
 'S7-i33-e40_2020-01-14-11-54-34.bag',
 'S1-i25-e35_2020-01-14-12-12-38.bag',
 'S3-i36-e15_2020-01-14-11-22-53.bag',
 'S6-i36-e15_2020-01-14-11-50-02.bag',
 'S3-i36-e15_2020-01-14-13-56-39.bag']

In [8]:
# In[9]:


valid_sim_bags = [bag for bag in sim_bags if bag not in failed_bags]
valid_sim_bags

['S7-i36-e15_2020-01-14-11-59-06.bag',
 'S1-i25-e59_2020-01-14-12-17-09.bag',
 'S6-i36-e15_2020-01-14-14-23-48.bag',
 'S1-i25-e59_2020-01-14-13-33-53.bag',
 'S2-i25-e35_2020-01-14-12-26-11.bag',
 'S4-i36-e15_2020-01-14-12-48-47.bag',
 'S2-i25-e35_2020-01-14-13-42-56.bag',
 'S1-i25-e35_2020-01-14-13-29-22.bag',
 'S2-i25-e59_2020-01-14-12-30-42.bag',
 'S2-i25-e59_2020-01-14-11-13-34.bag',
 'S3-i33-e40_2020-01-14-13-51-59.bag',
 'S4-i33-e40_2020-01-14-12-44-15.bag',
 'S7-i33-e40_2020-01-14-14-28-18.bag',
 'S1-i25-e35_2020-01-14-10-54-56.bag',
 'S7-i33-e40_2020-01-14-13-11-18.bag',
 'S4-i36-e15_2020-01-14-14-05-43.bag',
 'S7-i36-e15_2020-01-14-14-32-49.bag',
 'S2-i25-e59_2020-01-14-13-47-27.bag',
 'S7-i36-e15_2020-01-14-13-15-48.bag',
 'S6-i33-e40_2020-01-14-11-45-31.bag',
 'S1-i25-e59_2020-01-14-11-00-00.bag',
 'S3-i33-e40_2020-01-14-11-18-05.bag',
 'S3-i33-e40_2020-01-14-12-35-13.bag',
 'S4-i33-e40_2020-01-14-14-01-10.bag',
 'S6-i36-e15_2020-01-14-13-06-47.bag',
 'S2-i25-e35_2020-01-14-1

In [9]:
# In[ ]:


len(valid_sim_bags)

30

In [10]:
# In[8]:


pc_bag_codes = [
    "S1-i25-e35",
    "S1-i25-e59",
    "S2-i25-e35",
    "S2-i25-e59"
]

ot_bag_codes = [
    "S3-i33-e40",
    "S3-i36-e15",
    "S4-i33-e40",
    "S4-i36-e15"
]

pb_bag_codes = [
    "S6-i33-e40",
    "S6-i36-e15",
    "S7-i33-e40",
    "S7-i36-e15"
]

pc_bags = [bag for bag in valid_sim_bags if bag[:10] in pc_bag_codes]
ot_bags = [bag for bag in valid_sim_bags if bag[:10] in ot_bag_codes]
pb_bags = [bag for bag in valid_sim_bags if bag[:10] in pb_bag_codes]

In [11]:
# Get QTC_C sequence from bag
def qtccs_from_bag(bag):
    qtc_seq = []
    
    world = World_Trace()

    h_x = []
    h_y = []
    h_t = []
    r_x = []
    r_y = []
    r_t = []
    
    for topic, msg, t in bag.read_messages(topics=['/robot4/robot_pose']):
        r_x.append(msg.pose.position.x)
        r_y.append(msg.pose.position.y)
        t = t.to_sec()
        r_t.append(t)
    
    for topic, msg, t in bag.read_messages(topics=['/robot4/human_perception/tracked_persons']):
        try:
            h_x.append(msg.tracks[0].pose.pose.position.x)
            h_y.append(msg.tracks[0].pose.pose.position.y)
            t = t.to_sec()
            h_t.append(t)
        except:
            pass    


    # Downsample state series' to 200kHz frequency
    h_state_series = pd.DataFrame({"x": h_x, "y": h_y},
        index=[pd.to_datetime(h_t[i], unit="s") for i in range(len(h_t))])

    h_state_series = h_state_series.resample("200ms").mean()

    
    r_state_series = pd.DataFrame({"x": r_x, "y": r_y},
        index=[pd.to_datetime(r_t[i], unit="s") for i in range(len(r_t))])

    r_state_series = r_state_series.resample("200ms").mean()
    
    # Cut before and after human detection timestamps
    r_state_series = r_state_series.loc[h_state_series.index[0]:]
#     print(r_state_series)
#     print(h_state_series)
    h_state_series = h_state_series.loc[:r_state_series.index[-1]]

    # Create world_trace state series from downsampled human position data
    h_state_seq = []
    for index, row in h_state_series.iterrows():
        x = row['x']
        y = row['y']
        t = (pd.to_datetime(index) - dt.datetime(1970,1,1)).total_seconds()
#         print(t)

        h_state_seq.append(Object_State(name="human", timestamp=t, x=x, y=y))
        
        
    # Create world_trace state series from downsampled robot position data
    r_state_seq = []
    for index, row in r_state_series.iterrows():
        x = row['x']
        y = row['y']
        t = (pd.to_datetime(index) - dt.datetime(1970,1,1)).total_seconds()
#         print(t)

        r_state_seq.append(Object_State(name="robot", timestamp=t, x=x, y=y))
        

    # Add human and robot trajectories to world
    world.add_object_state_series(h_state_seq)
    world.add_object_state_series(r_state_seq)

    # make a QSRlib request message
    dynamic_args = {"qtccs": {"no_collapse": True, "quantisation_factor": quantisation_factor,
                                            "validate": True, "qsrs_for": [("robot", "human")]}}

    qsrlib_request_message = QSRlib_Request_Message(
        'qtccs', world, dynamic_args)

    # request your QSRs
    qsrlib = QSRlib()
    qsrlib_response_message = qsrlib.request_qsrs(req_msg=qsrlib_request_message)
    qsrlib_response_message
    
    
    # Get QSR at each timestamp
    timestamps = qsrlib_response_message.qsrs.get_sorted_timestamps()
    for t in timestamps:
        for val in qsrlib_response_message.qsrs.trace[t].qsrs.values():
            if val.qsr['qtccs'].replace(",","") != "0000":
                qtc_seq.append(val.qsr['qtccs'].replace(",",""))
#     qtc_seqs[sit_code] = qtc_seq
    return qtc_seq

In [12]:
quantisation_factor = 0.02

In [13]:
# Get path-crossing QTC_C sequences
pc_seqs = []
for bag_path in pc_bags:
    bag_path = "sim_bags/" + bag_path
    bag = rosbag.Bag(bag_path)
    pc_seqs.append(qtccs_from_bag(bag))

In [14]:
pc_seqs

[['0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '0-0+',
  '--0+',
  '--0+',
  '--0+',
  '--0+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '---+',
  '-0-+',
  '-0-+',
  '-0-+',
  '-00+',
  '-00+',
  '-00+',
  '-00+',
  '-00+',
  '-00+',
  '-00+',
  '--0+',
  '--0+',
  '--0+',
  '--0+',
  '-00+',
  '-00+',
  '-00+',
  '-00+',
  '-00+',
  '-00+',


In [15]:
# Get overtaking QTC_C sequences

ot_seqs = []
for bag_path in ot_bags:
    bag_path = "sim_bags/" + bag_path
    bag = rosbag.Bag(bag_path)
    ot_seqs.append(qtccs_from_bag(bag))    

In [16]:
ot_seqs

[['000-',
  '000-',
  '000-',
  '000-',
  '000-',
  '000-',
  '000-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '0+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-++-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+00',
  '-+00',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+0-',
  '-+00',
  '-+00',
  '-+00',
  '-+00',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',
  '-000',


In [17]:
# Get pass-by QTC_C sequences
pb_seqs = []
for bag_path in pb_bags:
    bag_path = "sim_bags/" + bag_path
    bag = rosbag.Bag(bag_path)
    pb_seqs.append(qtccs_from_bag(bag))

In [18]:
# In[159]:


pb_seqs

[['0-00',
  '0-0-',
  '0-00',
  '0-00',
  '0-00',
  '0-00',
  '0-00',
  '0-00',
  '0-00',
  '0-00',
  '0-00',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-00',
  '0-00',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '0-0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '--0-',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '----',
  '0---',
  '0---',
  '00--',
  '00--',
  '0+--',
  '0+--',
  '0+--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '++--',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',
  '+0-0',


In [19]:
# In[167]:


np.save("pc_sim_seqs", pc_seqs)
np.save("ot_sim_seqs", ot_seqs)
np.save("pb_sim_seqs", pb_seqs)

In [20]:
# In[ ]:


np.load("pb_sim_seqs.npy", allow_pickle=True)

array([list(['0-00', '0-0-', '0-00', '0-00', '0-00', '0-00', '0-00', '0-00', '0-00', '0-00', '0-00', '0-0-', '0-0-', '0-0-', '0-0-', '0-0-', '0-0-', '0-00', '0-00', '0-0-', '0-0-', '0-0-', '0-0-', '0-0-', '0-0-', '0-0-', '0-0-', '--0-', '--0-', '--0-', '--0-', '--0-', '--0-', '--0-', '--0-', '--0-', '--0-', '--0-', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '----', '0---', '0---', '00--', '00--', '0+--', '0+--', '0+--', '++--', '++--', '++--', '++--', '++--', '++--', '++--', '++--', '++--', '++--', '++--', '++--', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+0-0', '+000', '+000', '+000', '+000', '+000', '+000', '+000', '+000', '+000', '+000', '+000', '+000', '+000', '+0